In [6]:
import pandas as pd
from scripts import project_functions

# Load data from National Names
national_df = project_functions.load_and_process_one('../../data/raw/NationalNames.csv')
# Load 4 parts of raw data on State Names
state_df = project_functions.load_and_process_many("../../data/raw")


In [7]:
national_df

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746
...,...,...,...,...,...
1825428,1825429,Zykeem,2014,M,5
1825429,1825430,Zymeer,2014,M,5
1825430,1825431,Zymiere,2014,M,5
1825431,1825432,Zyran,2014,M,5


In [8]:
state_df

,Id,Name,Year,Gender,Count,State
0,1,Mary,1880,F,7065,NaN
1,2,Anna,1880,F,2604,NaN
2,3,Emma,1880,F,2003,NaN
3,4,Elizabeth,1880,F,1939,NaN
4,5,Minnie,1880,F,1746,NaN
...,...,...,...,...,...,...
7472854,5647422,Seth,2014,M,5,WY
7472855,5647423,Spencer,2014,M,5,WY
7472856,5647424,Tyce,2014,M,5,WY
7472857,5647425,Victor,2014,M,5,WY
